### Querying dataframes with Apache Spark
This notebook contains sample code from Chapter 4 of [Learning PySpark]() focusing on PySpark and DataFrames with a little changes :)

### Generate your own DataFrame
Instead of accessing the file system, let's create a DataFrame by generating the data.  In this case, we'll first create the `stringRDD` RDD and then convert it into a DataFrame when we're reading `stringJSONRDD` using `spark.read.json`.

In [32]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [33]:
# Generate our own JSON data 
#   This way we don't have to access the file system yet.
stringJSONRDD = sc.parallelize((""" 
  { "id": "123",
    "name": "Kristen",
    "age": 19,
    "eyeColor": "brown"
  }""",
   """{
    "id": "234",
    "name": "Randy",
    "age": 22,
    "eyeColor": "green"
  }""", 
  """{
    "id": "345",
    "name": "Favio",
    "age": 23,
    "eyeColor": "blue"
  }""")
)

In [34]:
# Create DataFrame
swimmersJSON = spark.read.json(stringJSONRDD)

In [35]:
# Create temporary table
swimmersJSON.createOrReplaceTempView("swimmersJSON")

In [36]:
# DataFrame API
swimmersJSON.show()

+---+--------+---+-------+
|age|eyeColor| id|   name|
+---+--------+---+-------+
| 19|   brown|123|Kristen|
| 22|   green|234|  Randy|
| 23|    blue|345|  Favio|
+---+--------+---+-------+



In [37]:
# SQL Query
spark.sql("select * from swimmersJSON").collect()

[Row(age=19, eyeColor='brown', id='123', name='Kristen'),
 Row(age=22, eyeColor='green', id='234', name='Randy'),
 Row(age=23, eyeColor='blue', id='345', name='Favio')]

In [38]:
%load_ext sql_magic

The sql_magic extension is already loaded. To reload it, use:
  %reload_ext sql_magic


In [39]:
%config SQL.conn_name = 'spark'

In [40]:
%%read_sql
select * from swimmersJSON

Query started at 11:44:39 PM CDT; Query executed in 0.00 m

,age,eyeColor,id,name
0,19,brown,123,Kristen
1,22,green,234,Randy
2,23,blue,345,Favio


#### Inferring the Schema Using Reflection
Note that Apache Spark is inferring the schema using reflection; i.e. it automaticlaly determines the schema of the data based on reviewing the JSON data.

In [41]:
# Print the schema
swimmersJSON.printSchema()

root
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)



Notice that Spark was able to determine infer the schema (when reviewing the schema using `.printSchema`).

But what if we want to programmatically specify the schema?

#### Programmatically Specifying the Schema
In this case, let's specify the schema for a `CSV` text file.

In [42]:
from pyspark.sql.types import *

# Generate our own CSV data 
#   This way we don't have to access the file system yet.
stringCSVRDD = sc.parallelize([(123, 'Katie', 19, 'brown'), (234, 'Michael', 22, 'green'), (345, 'Simone', 23, 'blue')])

# The schema is encoded in a string, using StructType we define the schema using various pyspark.sql.types
schemaString = "id name age eyeColor"
schema = StructType([
    StructField("id", LongType(), True),    
    StructField("name", StringType(), True),
    StructField("age", LongType(), True),
    StructField("eyeColor", StringType(), True)
])

# Apply the schema to the RDD and Create DataFrame
swimmers = spark.createDataFrame(stringCSVRDD, schema)

# Creates a temporary view using the DataFrame
swimmers.createOrReplaceTempView("swimmers")

In [43]:
# Print the schema
#   Notice that we have redefined id as Long (instead of String)
swimmers.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)



In [44]:
%%read_sql
-- Query the data
select * from swimmers

Query started at 11:44:40 PM CDT; Query executed in 0.00 m

,id,name,age,eyeColor
0,123,Katie,19,brown
1,234,Michael,22,green
2,345,Simone,23,blue


As you can see from above, we can programmatically apply the `schema` instead of allowing the Spark engine to infer the schema via reflection.

Additional Resources include:
* [PySpark API Reference](https://spark.apache.org/docs/2.0.0/api/python/pyspark.sql.html)
* [Spark SQL, DataFrames, and Datasets Guide](https://spark.apache.org/docs/latest/sql-programming-guide.html#programmatically-specifying-the-schema): This is in reference to Programmatically Specifying the Schema using a `CSV` file.

### Querying with SQL
With DataFrames, you can start writing your queries using `Spark SQL` - a SQL dialect that is compatible with the Hive Query Language (or HiveQL).

In [45]:
# Execute SQL Query and return the data
spark.sql("select * from swimmers").show()

+---+-------+---+--------+
| id|   name|age|eyeColor|
+---+-------+---+--------+
|123|  Katie| 19|   brown|
|234|Michael| 22|   green|
|345| Simone| 23|    blue|
+---+-------+---+--------+



Let's get the row count:

In [46]:
# Get count of rows in SQL
spark.sql("select count(1) from swimmers").show()

+--------+
|count(1)|
+--------+
|       3|
+--------+



Note, you can make use of `%sql` within the notebook cells of a Databricks notebook.

In [47]:
%%read_sql
-- Query all data
select * from swimmers

Query started at 11:44:41 PM CDT; Query executed in 0.00 m

,id,name,age,eyeColor
0,123,Katie,19,brown
1,234,Michael,22,green
2,345,Simone,23,blue


In [48]:
# Query id and age for swimmers with age = 22 via DataFrame API
swimmers.select("id", "age").filter("age = 22").show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [49]:
# Query id and age for swimmers with age = 22 via DataFrame API in another way
swimmers.select(swimmers.id, swimmers.age).filter(swimmers.age == 22).show()


+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [50]:
# Query id and age for swimmers with age = 22 in SQL
spark.sql("select id, age from swimmers where age = 22").show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [51]:
%%read_sql
-- Query id and age for swimmers with age = 22
select id, age from swimmers where age = 22

Query started at 11:44:42 PM CDT; Query executed in 0.00 m

,id,age
0,234,22


In [52]:
# Query name and eye color for swimmers with eye color starting with the letter 'b'
spark.sql("select name, eyeColor from swimmers where eyeColor like 'b%'").show()

+------+--------+
|  name|eyeColor|
+------+--------+
| Katie|   brown|
|Simone|    blue|
+------+--------+



In [53]:
%%read_sql
-- Query name and eye color for swimmers with eye color starting with the letter 'b'
select name, eyeColor from swimmers where eyeColor like 'b%'

Query started at 11:44:42 PM CDT; Query executed in 0.00 m

,name,eyeColor
0,Katie,brown
1,Simone,blue


### Querying with the DataFrame API
With DataFrames, you can start writing your queries using the DataFrame API

In [54]:
# Show the values 
swimmers.show()

+---+-------+---+--------+
| id|   name|age|eyeColor|
+---+-------+---+--------+
|123|  Katie| 19|   brown|
|234|Michael| 22|   green|
|345| Simone| 23|    blue|
+---+-------+---+--------+



In [55]:
# Using Databricks `display` command to view the data easier
display(swimmers)

DataFrame[id: bigint, name: string, age: bigint, eyeColor: string]

In [56]:
# Get count of rows
swimmers.count()

3

In [57]:
# Get the id, age where age = 22
swimmers.select("id", "age").filter("age = 22").show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [58]:
# Get the name, eyeColor where eyeColor like 'b%'
swimmers.select("name", "eyeColor").filter("eyeColor like 'b%'").show()

+------+--------+
|  name|eyeColor|
+------+--------+
| Katie|   brown|
|Simone|    blue|
+------+--------+



## On-Time Flight Performance
Query flight departure delays by State and City by joining the departure delay and join to the airport codes (to identify state and city).

### DataFrame Queries
Let's run a flight performance using DataFrames; let's first build the DataFrames from the source datasets.

In [59]:
# Set File Paths
flightPerfFilePath = "flight-data/departuredelays.csv"
airportsFilePath = "flight-data/airport-codes-na.txt"

# Obtain Airports dataset
airports = spark.read.csv(airportsFilePath, header='true', inferSchema='true', sep='\t')
airports.createOrReplaceTempView("airports")

# Obtain Departure Delays dataset
flightPerf = spark.read.csv(flightPerfFilePath, header='true')
flightPerf.createOrReplaceTempView("FlightPerformance")

# Cache the Departure Delays dataset 
flightPerf.cache()

DataFrame[date: string, delay: string, distance: string, origin: string, destination: string]

In [60]:
# Query Sum of Flight Delays by City and Origin Code (for Washington State)
spark.sql("select a.City, f.origin, sum(f.delay) as Delays from FlightPerformance f join airports a on a.IATA = f.origin where a.State = 'WA' group by a.City, f.origin order by sum(f.delay) desc").show()

+-------+------+--------+
|   City|origin|  Delays|
+-------+------+--------+
|Seattle|   SEA|159086.0|
|Spokane|   GEG| 12404.0|
|  Pasco|   PSC|   949.0|
+-------+------+--------+



In [61]:
%%read_sql
-- Query Sum of Flight Delays by City and Origin Code (for Washington State)
select a.City, f.origin, sum(f.delay) as Delays
  from FlightPerformance f
    join airports a
      on a.IATA = f.origin
 where a.State = 'WA'
 group by a.City, f.origin
 order by sum(f.delay) desc

Query started at 11:44:45 PM CDT; Query executed in 0.02 m

,City,origin,Delays
0,Seattle,SEA,159086.0
1,Spokane,GEG,12404.0
2,Pasco,PSC,949.0


In [62]:
# Query Sum of Flight Delays by State (for the US)
spark.sql("select a.State, sum(f.delay) as Delays from FlightPerformance f join airports a on a.IATA = f.origin where a.Country = 'USA' group by a.State ").show()

+-----+---------+
|State|   Delays|
+-----+---------+
|   SC|  80666.0|
|   AZ| 401793.0|
|   LA| 199136.0|
|   MN| 256811.0|
|   NJ| 452791.0|
|   OR| 109333.0|
|   VA|  98016.0|
| null| 397237.0|
|   RI|  30760.0|
|   WY|  15365.0|
|   KY|  61156.0|
|   NH|  20474.0|
|   MI| 366486.0|
|   NV| 474208.0|
|   WI| 152311.0|
|   ID|  22932.0|
|   CA|1891919.0|
|   CT|  54662.0|
|   NE|  59376.0|
|   MT|  19271.0|
+-----+---------+
only showing top 20 rows



In [63]:
%%read_sql
-- Query Sum of Flight Delays by State (for the US)
select a.State, sum(f.delay) as Delays
  from FlightPerformance f
    join airports a
      on a.IATA = f.origin
 where a.Country = 'USA'
 group by a.State 

Query started at 11:44:46 PM CDT; Query executed in 0.01 m

,State,Delays
0,SC,80666.0
1,AZ,401793.0
2,LA,199136.0
3,MN,256811.0
4,NJ,452791.0
5,OR,109333.0
6,VA,98016.0
7,None,397237.0
8,RI,30760.0
9,WY,15365.0


In [64]:
%%read_sql
-- Query Sum of Flight Delays by State (for the US)
select a.State, sum(f.delay) as Delays
  from FlightPerformance f
    join airports a
      on a.IATA = f.origin
 where a.Country = 'USA'
 group by a.State 

Query started at 11:44:47 PM CDT; Query executed in 0.01 m

,State,Delays
0,SC,80666.0
1,AZ,401793.0
2,LA,199136.0
3,MN,256811.0
4,NJ,452791.0
5,OR,109333.0
6,VA,98016.0
7,None,397237.0
8,RI,30760.0
9,WY,15365.0
